In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import json
import yaml

from IPython.display import display, Markdown

import numpy as np
import pandas as pd

from scipy.interpolate import interp1d
# from decode_trf import delivery_data_from_logfile

from pymedphys.msq import mosaiq_connect
from pymedphys.utilities import get_index, get_data_directory, get_filepath, get_gantry_tolerance
from pymedphys.logfile import *
from pymedphys.trf import *

from pymedphys.mudensity import *

from pymedphys.plt import pcolormesh_grid

In [ ]:
with open('../config.json') as config_file:
    config = json.load(config_file)

In [ ]:
index = get_index(config)

In [ ]:
data_directory = get_data_directory(config)

cache_filepath = os.path.join(data_directory, 'cache', 'dmlc_comparison.yaml')
cache_scratch_filepath = os.path.join(data_directory, 'cache', 'dmlc_comparison_scratch.yaml')

with open(cache_filepath, 'r') as cache_file:
    cache = yaml.load(cache_file)

In [ ]:
patient_ids = list(cache.keys())
len(patient_ids)

In [ ]:
data = []

for patient_id in patient_ids:
    for byfield in cache[patient_id]:
        for angle in cache[patient_id][byfield]:
#             print(cache[patient_id][byfield][angle].keys())
            comparison = cache[patient_id][byfield][angle]['median']
            file_hashes = cache[patient_id][byfield][angle]['median_filehash_group']
            all_comparisons = cache[patient_id][byfield][angle]['comparisons']
            all_file_hashes = cache[patient_id][byfield][angle]['filehash_groups']
            all_comparisons_flat = []
            for key, value in all_comparisons.items():
                all_comparisons_flat.append(value)
            
            data.append([
                patient_id, byfield, angle, comparison, file_hashes, tuple(all_comparisons_flat), tuple(all_file_hashes)
            ])
            
comparisons_table = pd.DataFrame(
    columns=[
        'patient_id', 'field_id',
        'gantry_angle', 'comparison',
        'file_hashes', 'all_comparisons',
        'all_file_hashes'
    ],
    data=data
)

comparisons_table = comparisons_table.sort_values('comparison', ascending=False)
top_ten = comparisons_table.iloc[0:10]
top_ten

In [ ]:
field_id = 77630

In [ ]:
field_ref = comparisons_table['field_id'] == field_id
comparisons_table[field_ref]

In [ ]:
worst_row_of_field = comparisons_table[field_ref].iloc[0]

In [ ]:
gantry_angle = worst_row_of_field['gantry_angle']
gantry_angle

In [ ]:
gantry_angles = comparisons_table[field_ref]['gantry_angle'].values
gantry_angles

In [ ]:
worst_row_of_field['all_file_hashes']

In [ ]:
worst_file_ref = np.argmax(worst_row_of_field['all_comparisons'])
logfile_group = worst_row_of_field['all_file_hashes'][worst_file_ref]
logfile_group

In [ ]:
file_hash = logfile_group[0]
file_hash

In [ ]:
# with mosaiq_connect('msqsql') as cursor:
#     comparison = compare_logfile_group_bygantry(
#         index, config, cursor, logfile_group, gantry_angle)

In [ ]:
filepath = get_filepath(index, config, file_hash)
filepath

In [ ]:
logfile_delivery_data = delivery_data_from_logfile(filepath)
full_logfile = decode_trf(filepath)

In [ ]:
len(logfile_delivery_data.monitor_units)

In [ ]:
control_point_label = full_logfile.columns[0]
control_point_label

In [ ]:

mu_label = 'Step Dose/Actual Value (Mu)'

In [ ]:
np.cumsum([0,1,2,0,0,2])

In [ ]:
diff = np.diff(
    np.concatenate([[0], full_logfile[mu_label].values])
)

diff[diff < 0] = 0

mu_test = np.cumsum(diff)

In [ ]:
# mu_test = np.cumsum(
#     np.diff(
#         np.concatenate([[0], full_logfile[mu_label].values])
#     )
# # )

In [ ]:
moniter_units = np.array(logfile_delivery_data.monitor_units)
moniter_units

In [ ]:
plt.plot(mu_test)
plt.plot(moniter_units)

In [ ]:
np.allclose(mu_test, moniter_units)

In [ ]:
np.sum(find_relevant_control_points(moniter_units))

In [ ]:
jaw_pos_error_label = 'X1 Diaphragm/Positional Error (mm)'

In [ ]:
jaw_pos_error[3249]

In [ ]:
full_logfile[mu_label].values[3249]

In [ ]:
# jaw_pos_error_label = 'Dlg Y2/Positional Error (mm)'

# gantry_angles = np.array(delivery_data.gantry)
# jaw_pos_error = full_logfile[jaw_pos_error_label].values

# gantry_angle_within_tolerance = (
#     np.abs(gantry_angles - gantry_angle) <= gantry_tolerance)

# np.max(np.abs(jaw_pos_error[gantry_angle_within_tolerance]))

In [ ]:
delivery_data = logfile_delivery_data
gantry_tolerance = 0.5


moniter_units = np.array(delivery_data.monitor_units)
relevant_control_points = find_relevant_control_points(moniter_units)

mu = moniter_units[relevant_control_points]
mlc = np.array(delivery_data.mlc)[relevant_control_points]
jaw = np.array(delivery_data.jaw)[relevant_control_points]
gantry_angles = np.array(delivery_data.gantry)[relevant_control_points]

control_points = full_logfile[control_point_label].values[relevant_control_points]
jaw_pos_error = full_logfile[jaw_pos_error_label].values[relevant_control_points]

gantry_angle_within_tolerance = (
    np.abs(gantry_angles - gantry_angle) <= gantry_tolerance)
diff_mu = np.concatenate([[0], np.diff(mu)])[gantry_angle_within_tolerance]
mu = np.cumsum(diff_mu)

mlc = mlc[gantry_angle_within_tolerance]
jaw = jaw[gantry_angle_within_tolerance]

control_points = control_points[gantry_angle_within_tolerance]
jaw_pos_error = jaw_pos_error[gantry_angle_within_tolerance]


logfile_mu, logfile_mlc, logfile_jaw = mu, mlc, jaw

In [ ]:
# logfile_mu, logfile_mlc, logfile_jaw = extract_angle_from_delivery_data(logfile_delivery_data, gantry_angle, gantry_tolerance=0.5)

In [ ]:
with mosaiq_connect('msqsql') as cursor:
    mosaiq_delivery_data = multi_fetch_and_verify_mosaiq(cursor, field_id)
    
mosaiq_mu, mosaiq_mlc, mosaiq_jaw = extract_angle_from_delivery_data(mosaiq_delivery_data, gantry_angle)

In [ ]:
# logfile_delivery_data

In [ ]:
grid_resolution = 0.25

In [ ]:
# mosaiq_mu_density = calc_mu_density_bygantry(
#     mosaiq_delivery_data, gantry_angle, grid_resolution)

# normalisation = calc_normalisation(mosaiq_delivery_data)

# logfile_mu_density = calc_logfile_mu_density_bygantry(
#     index, config, logfile_group, gantry_angle, grid_resolution)

# grid_xx = logfile_mu_density[0]
# grid_yy = logfile_mu_density[1]

# logfile_mu_density = logfile_mu_density[2]
# mosaiq_mu_density = mosaiq_mu_density[2]

In [ ]:
save_file_path = r'S:\Physics\Programming\data\LinacLogFiles\results\EPSM2018_77630_investigation.json'


with open(save_file_path, 'r') as save_file:
    loaded_data = json.load(save_file)
    
x = np.array(loaded_data['x'])
y = np.array(loaded_data['y'])

logfile_mu_density = np.array(loaded_data['logfile'])
mosaiq_mu_density = np.array(loaded_data['mosaiq'])

In [ ]:
min_val = np.min([logfile_mu_density, mosaiq_mu_density])
max_val = np.max([logfile_mu_density, mosaiq_mu_density])

In [ ]:
# x = grid_xx[0,:]
# y = grid_yy[:,0]

# x, y = pcolormesh_grid(x, y)

# x = -x

In [ ]:
# x

In [ ]:
# to_save = {
#     'x': x.tolist(),
#     'y': y.tolist(),
#     'logfile': np.round(logfile_mu_density, 2).tolist(),
#     'mosaiq': np.round(mosaiq_mu_density, 2).tolist()
# }

In [ ]:
# save_file_path = r'S:\Physics\Programming\data\LinacLogFiles\results\EPSM2018_77630_investigation.json'

In [ ]:
# with open(save_file_path, 'w') as save_file:
#     json.dump(to_save, save_file)

In [ ]:
figsize = (6.5, 9.5)

In [ ]:
plt.figure(figsize=figsize)
plt.pcolormesh(x, y, logfile_mu_density, vmin=min_val, vmax=max_val)
plt.colorbar(label='MU density')
plt.title('Logfile MU density')
plt.xlabel('MLC direction (mm)')
plt.ylabel('Jaw direction (mm)')

plt.axis('equal')

plt.xlim([-67, 60])
plt.ylim([60, -75])

plt.savefig('logfile.png')

In [ ]:
plt.figure(figsize=figsize)
plt.pcolormesh(x, y, mosaiq_mu_density, vmin=min_val, vmax=max_val)
plt.colorbar(label='MU density')
plt.title('Mosaiq MU density')
plt.xlabel('MLC direction (mm)')
plt.ylabel('Jaw direction (mm)')

plt.axis('equal')

plt.xlim([-67, 60])
plt.ylim([60, -75])

plt.savefig('mosaiq.png')

In [ ]:
difference = logfile_mu_density - mosaiq_mu_density
max_range = np.max(np.abs(difference))

In [ ]:
plt.figure(figsize=figsize)
plt.pcolormesh(x, y, difference, vmin=-max_range, vmax=max_range, cmap='bwr')
plt.colorbar(label='Logfile - Mosaiq MU density')
plt.title('MU density difference')
plt.xlabel('MLC direction (mm)')
plt.ylabel('Jaw direction (mm)')

plt.axis('equal')

plt.xlim([-67, 60])
plt.ylim([60, -75])

plt.savefig('diff.png')

In [ ]:
figsize2 = (8.5, 9.5)


plt.figure(figsize=figsize2)
plt.pcolormesh(x, y, difference, vmin=-max_range, vmax=max_range, cmap='bwr')
plt.colorbar(label='Logfile - Mosaiq MU density')
plt.title('MU density difference')
plt.xlabel('MLC direction (mm)')
plt.ylabel('Jaw direction (mm)')

plt.axis('equal')

plt.xlim([-67, 60])
plt.ylim([60, -75])

plt.savefig('diff.png')

In [ ]:
np.shape(mosaiq_mu)

In [ ]:
np.shape(mosaiq_jaw)

In [ ]:
new_control_point = logfile_mu[np.diff(np.concatenate([[-1], control_points])) != 0]
new_control_point

In [ ]:
jaw_interp = interp1d(mosaiq_mu, mosaiq_jaw[:,0])
mosaiq_resampled = jaw_interp(logfile_mu)


np.max(np.abs(logfile_jaw[:,0] - mosaiq_resampled))

In [ ]:
np.max(np.abs(jaw_pos_error))

In [ ]:
plt.plot(mosaiq_mu, mosaiq_jaw[:,0])
plt.plot(logfile_mu, logfile_jaw[:,0])

# plt.plot(logfile_mu, control_points)

for mu in new_control_point:
    plt.plot([mu,mu], [40,65], 'k--', alpha=0.2)
    
plt.ylim([44,61])

In [ ]:
plot_new_control_point = np.concatenate([new_control_point, [np.max(mosaiq_mu)]])

In [ ]:
jaw_pos_error

In [ ]:
np.max(np.abs(jaw_pos_error))

In [ ]:
plt.figure(figsize=figsize)

plt.title('Jaw vs MU')
plt.xlabel('Cumulative Monitor Units')
plt.ylabel('Jaw position (mm)')

plt.plot(mosaiq_mu, mosaiq_jaw[:,0], label='Mosaiq Record')
plt.plot(logfile_mu, logfile_jaw[:,0], label='Logfile Record')

plt.plot(logfile_mu, logfile_jaw[:,0] + jaw_pos_error, label='Logfile Record')

for mu in plot_new_control_point:
    plt.plot([mu,mu], [40,65], 'k--', alpha=0.2)

plt.plot([mu,mu], [100,100], 'k--', alpha=0.2, label='Control Point Boundaries')
# plt.plot([mu2,mu2], [40,65], 'k-', alpha=0.5)

plt.ylim([44,61])

plt.legend()

plt.savefig('jaw.png')

plt.show()

In [ ]:


# plt.plot(logfile_mu, control_points)

# for mu in new_control_point:
#     plt.plot([mu,mu], [40,65], 'k--', alpha=0.2)
    


for i, (mu1, mu2) in enumerate(zip(plot_new_control_point[0:-1], plot_new_control_point[1::])):
    plt.figure()
    
    plt.title('Jaw vs MU')
    plt.xlabel('Cumulative Monitor Units')
    plt.ylabel('Jaw position (mm)')
    
    plt.plot(mosaiq_mu, mosaiq_jaw[:,0])
    plt.plot(logfile_mu, logfile_jaw[:,0])
    
    for mu in plot_new_control_point:
        plt.plot([mu,mu], [40,65], 'k--', alpha=0.2)
    
    plt.plot([mu1,mu1], [40,65], 'k-', alpha=0.5)
    plt.plot([mu2,mu2], [40,65], 'k-', alpha=0.5)
    
    plt.ylim([44,61])
    
    plt.savefig('{}_jaw.png'.format(i))
    
    plt.show()

In [ ]:
len(new_control_point)

In [ ]:
np.sort(np.unique(control_points))

In [ ]:
# slice_ref = control_point == control_points
# index_ref = np.where(slice_ref)[0]

# index_ref

In [ ]:
# slice_ref = control_point == control_points
# index_ref = np.where(slice_ref)[0]
# if index_ref[0] != 0:
#     index_ref = np.concatenate([[index_ref[0] - 1], index_ref])
    
# index_ref

In [ ]:
mosaiq_cp_mu = []
logfile_cp_mu = []


for i, control_point in enumerate(np.sort(np.unique(control_points))):
    a_slice = slice(i, i + 2, 1)
    slice_ref = control_point == control_points
    index_ref = np.where(slice_ref)[0]
    if index_ref[0] != 0:
        index_ref = np.concatenate([[index_ref[0] - 1], index_ref])
    
    mosaiq_cp_mu.append(np.max(mosaiq_mu[a_slice]) - np.min(mosaiq_mu[a_slice]))
    logfile_cp_mu.append(np.max(logfile_mu[index_ref]) - np.min(logfile_mu[index_ref]))

    
plt.plot(mosaiq_cp_mu)
plt.plot(logfile_cp_mu)

In [ ]:
mu_density_by_slice = []

for i, control_point in enumerate(np.sort(np.unique(control_points))):
    results = dict()
    
    a_slice = slice(i, i + 2, 1)
    slice_ref = control_point == control_points
    index_ref = np.where(slice_ref)[0]
    if index_ref[0] != 0:
        index_ref = np.concatenate([[index_ref[0] - 1], index_ref])
    
    results['mosaiq'] = calc_mu_density(
        mosaiq_mu[a_slice], mosaiq_mlc[a_slice, :, :], 
        mosaiq_jaw[a_slice, :], grid_resolution=grid_resolution)

    results['logfile'] = calc_mu_density(
        logfile_mu[index_ref], logfile_mlc[index_ref, :, :],
        logfile_jaw[index_ref, :], grid_resolution=grid_resolution)
    
    mu_density_by_slice.append(results)

In [ ]:
# a_slice = slice(0, 2, 1)

# mu = mosaiq_mu[a_slice]
# mlc = mosaiq_mlc[a_slice, :, :]
# jaw = mosaiq_jaw[a_slice, :]

# mosaiq_control_point_mu_density = calc_mu_density(mu, mlc, jaw, grid_resolution=grid_resolution)

In [ ]:
# slice_ref = control_points[0] == control_points

# mu = logfile_mu[slice_ref]
# mlc = logfile_mlc[slice_ref, :, :]
# jaw = logfile_jaw[slice_ref, :]

# logfile_control_point_mu_density = calc_mu_density(mu, mlc, jaw, grid_resolution=grid_resolution)

In [ ]:
# mosaiq_control_point_mu_density

In [ ]:
max_all = np.array([
    np.max([a_mu_slice['logfile'], a_mu_slice['mosaiq']])
    for a_mu_slice in mu_density_by_slice
])


In [ ]:
max_val = np.max(max_all)

In [ ]:
differences = [
    a_mu_slice['logfile'] - a_mu_slice['mosaiq']
    for a_mu_slice in mu_density_by_slice
]

In [ ]:
max_diff_per_control_point = np.max(np.abs(differences))

In [ ]:
def plot_and_save_per_control_point(i):
    logfile_control_point_mu_density = mu_density_by_slice[i]['logfile']
    mosaiq_control_point_mu_density = mu_density_by_slice[i]['mosaiq']
    slice_difference = differences[i]

    figsize = (6.5, 9.5)


    plt.figure(figsize=figsize)
    plt.pcolormesh(x, y, logfile_control_point_mu_density, vmin=0, vmax=max_val)
    plt.colorbar(label='MU density')
    plt.title('Logfile MU density')
    plt.xlabel('MLC direction (mm)')
    plt.ylabel('Jaw direction (mm)')

    plt.axis('equal')

    plt.xlim([-67, 60])
    plt.ylim([60, -75])

    plt.savefig("{}_logfile.png".format(i))

    plt.show()


    plt.figure(figsize=figsize)
    plt.pcolormesh(x, y, mosaiq_control_point_mu_density, vmin=0, vmax=max_val)
    plt.colorbar(label='MU density')
    plt.title('Mosaiq MU density')
    plt.xlabel('MLC direction (mm)')
    plt.ylabel('Jaw direction (mm)')

    plt.axis('equal')

    plt.xlim([-67, 60])
    plt.ylim([60, -75])

    plt.savefig("{}_mosaiq.png".format(i))

    plt.show()


    plt.figure(figsize=figsize)
    plt.pcolormesh(
        x, y, slice_difference, 
        vmin=-max_diff_per_control_point, vmax=max_diff_per_control_point,
        cmap='bwr'
    )
    plt.colorbar(label='Logfile - Mosaiq MU density')
    plt.title('MU density difference')
    plt.xlabel('MLC direction (mm)')
    plt.ylabel('Jaw direction (mm)')

    plt.axis('equal')

    plt.xlim([-67, 60])
    plt.ylim([60, -75])

    plt.savefig("{}_diff.png".format(i))

    plt.show()

In [ ]:
for i in range(19):
    plot_and_save_per_control_point(i)



In [ ]:
# plot_and_save_per_control_point(1)

In [ ]:
plt.plot(mosaiq_mu, mosaiq_jaw[:,1])
plt.plot(logfile_mu, logfile_jaw[:,1])

In [ ]:
# logfile_delivery_data

In [ ]:
control_points

In [ ]:
control_points